In [1]:
import numpy as np
import pickle
import os
from src.ann_module import ANN
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score

### Data Imports and recollection

In [2]:
dir_path = r'./data/features/lab_single'
source_file_nr = len([entry for entry in os.listdir(dir_path) if os.path.isfile(os.path.join(dir_path, entry))])


for number in range(source_file_nr):
    
    with open(os.path.join(dir_path, f'onset_scenario_{number + 1}'), 'rb') as file:
        signal_extracted_scenario = pickle.load(file)
    # create a single array with all the feature files
    if number == 0:
        signal_extracted = signal_extracted_scenario
    else:
        signal_extracted = np.append(signal_extracted, signal_extracted_scenario, axis=0)

In [3]:
dir_path_multi = r'./data/features/lab_multi'
test_file_names = ['onset_scenario_3', 'onset_scenario_9', 'onset_scenario_12']
test_cases_names = ['A', 'B', 'C']
test_cases = {}
pd_clusters = [[2], [3], [3, 4, 5]]


for i, test_f in enumerate(test_file_names):
    
    with open(os.path.join(dir_path_multi, test_f), 'rb') as file:
        signal_extracted_scenario = pickle.load(file)
        
        # Cluster labels from clustering are unordered and do not correspond to the acual class. The proper PD-int class is assigned by study of the PRPD
        cluster_labels_old = signal_extracted_scenario[:, 2].copy()
        cluster_labels_new = np.ones(len(cluster_labels_old))
        cluster_labels_new[np.isin(cluster_labels_old, pd_clusters[i])] = 0

        signal_extracted_scenario[:, 2] = cluster_labels_new
        test_cases[test_cases_names[i]] = signal_extracted_scenario

Separate data

In [4]:
np.random.shuffle(signal_extracted)

# Unpack the data
id_train = signal_extracted[:, 0]
sens_train = signal_extracted[:, 1]
cluster_train = signal_extracted[:, 2].copy()
signals_train = signal_extracted[:, 3:]

Fix the labels 
- for training the labels should start from 0 and they are currently skipping value 7
- for test the values are labeled as clustering output (ex 1-5 not the recorded defect class)

- Scenario 3 - PD = cluster 1
- Scenario 9 - PD = cluster 2
- Scenario 12 - PD = cluster 3, 4


In [5]:
cluster_train[cluster_train > 6] = cluster_train[cluster_train > 6] - 1
cluster_train = cluster_train - 1
sensors_sorted = np.sort(np.unique(sens_train))

### Binary

In [6]:
y_train = np.zeros(np.shape(cluster_train))
y_train[cluster_train > 5] = 1  # the last few labels are interference (1) the first few PD (0)

# scaler = StandardScaler()
# X_train = scaler.fit_transform(signals_train)

X_train = signals_train

In [7]:
best_models_bin = []
results_bin = []

# One model per sensor is trained
for sensor in sensors_sorted:

    print(f'Train sens: {sensor}')
    X_train_sens = X_train[sens_train == sensor]
    y_train_sens = y_train[sens_train == sensor]

    ann_classifier = ANN(np.shape(X_train)[1], 25, len(np.unique(y_train)))
    ann_classifier.cv_fit(X_train_sens, y_train_sens, patience = 10)

    best_models_bin.append(ann_classifier)
    results_bin.append(ann_classifier.val_score_)


Train sens: 0
Training model with cv = 5 splits
Epoch 1/300
200/200 [==============================] - 1s 3ms/step - loss: 4.0619 - accuracy: 0.9353 - val_loss: 1.1253 - val_accuracy: 0.9612
Epoch 2/300
200/200 [==============================] - 0s 2ms/step - loss: 0.4801 - accuracy: 0.9812 - val_loss: 0.9319 - val_accuracy: 0.9712
Epoch 3/300
200/200 [==============================] - 0s 2ms/step - loss: 0.1579 - accuracy: 0.9892 - val_loss: 0.7034 - val_accuracy: 0.9774
Epoch 4/300
200/200 [==============================] - 0s 2ms/step - loss: 0.1477 - accuracy: 0.9904 - val_loss: 0.6076 - val_accuracy: 0.9818
Epoch 5/300
200/200 [==============================] - 0s 2ms/step - loss: 0.0948 - accuracy: 0.9933 - val_loss: 0.7639 - val_accuracy: 0.9825
Epoch 6/300
200/200 [==============================] - 0s 2ms/step - loss: 0.0814 - accuracy: 0.9942 - val_loss: 0.5079 - val_accuracy: 0.9862
Epoch 7/300
200/200 [==============================] - 0s 2ms/step - loss: 0.0403 - accuracy: 

In [8]:
for case in test_cases_names:

    data_case = test_cases[case]

    id_test = data_case[:, 0]
    sens_test = data_case[:, 1]
    cluster_test = data_case[:, 2]
    signals_test = data_case[:, 3:]
    

    # Scale the input and define binary label
    # scaler = StandardScaler()
    # X_test = scaler.fit_transform(signals_test)

    X_test = signals_test
    y_true = cluster_test

    # Create merged label by sensor (average target class for all events belonging to the same instance)
    unique_inst_id = np.sort(np.unique(id_test))
    y_true_merged = [round(np.average(y_true[id_test == id])) for id in unique_inst_id]

    id_list = []
    y_list = []

    # Iterate through sensors
    for sens in sensors_sorted:

        # Extract X and id for the tried sensor
        X_test_sens = X_test[sens_test == sens]
        id_test_sens = id_test[sens_test == sens]

        # Predict
        y_test_sens = best_models_bin[sens].predict(X_test_sens)

        id_list.extend(id_test_sens)
        y_list.extend(y_test_sens)


    id_array = np.array(id_list)
    y_array = np.array(y_list)
    y_test_merged = [round(np.average(y_array[id_array == id])+0.01) for id in unique_inst_id]

    print(f'Accuracy for case: {case}: {accuracy_score(y_true_merged, y_test_merged)}')
    # print(confusion_matrix(y_true_merged, y_test_merged))
    # print(recall_score(y_true_merged, y_test_merged))

Accuracy for case: A: 0.899812734082397
Accuracy for case: B: 0.961218836565097
Accuracy for case: C: 0.7989080982711556


### Multiclass

In [14]:
y_train = cluster_train  # the last few labels are interference (1) the first few PD (0)

# scaler = StandardScaler()
# X_train = scaler.fit_transform(signals_train)
X_train = signals_train

In [18]:
best_models_mc = []
results_mc = []

# One model per sensor is trained
for sensor in sensors_sorted:

    print(f'Train sens: {sensor}')
    X_train_sens = X_train[sens_train == sensor]
    y_train_sens = y_train[sens_train == sensor]

    ann_classifier = ANN(np.shape(X_train)[1], 60, len(np.unique(y_train)))
    ann_classifier.cv_fit(X_train_sens, y_train_sens, patience = 10)

    best_models_mc.append(ann_classifier)
    results_mc.append(ann_classifier.val_score_)

Train sens: 0
Training model with cv = 5 splits
Epoch 1/300
200/200 [==============================] - 1s 3ms/step - loss: 29.3770 - accuracy: 0.7621 - val_loss: 5.0593 - val_accuracy: 0.8716
Epoch 2/300
200/200 [==============================] - 0s 2ms/step - loss: 3.4954 - accuracy: 0.9038 - val_loss: 3.3262 - val_accuracy: 0.9079
Epoch 3/300
200/200 [==============================] - 0s 2ms/step - loss: 1.9600 - accuracy: 0.9326 - val_loss: 2.5533 - val_accuracy: 0.9110
Epoch 4/300
200/200 [==============================] - 0s 2ms/step - loss: 1.3125 - accuracy: 0.9397 - val_loss: 2.5631 - val_accuracy: 0.9135
Epoch 5/300
200/200 [==============================] - 0s 2ms/step - loss: 1.0065 - accuracy: 0.9489 - val_loss: 2.3032 - val_accuracy: 0.9217
Epoch 6/300
200/200 [==============================] - 0s 2ms/step - loss: 0.7691 - accuracy: 0.9531 - val_loss: 2.1866 - val_accuracy: 0.9142
Epoch 7/300
200/200 [==============================] - 0s 2ms/step - loss: 0.7448 - accuracy:

In [19]:
for case in test_cases_names:

    data_case = test_cases[case]

    id_test = data_case[:, 0]
    sens_test = data_case[:, 1]
    cluster_test = data_case[:, 2]
    signals_test = data_case[:, 3:]
    

    # Scale the input and define binary label
    # scaler = StandardScaler()
    # X_test = scaler.fit_transform(signals_test)
    X_test = signals_test


    y_true = cluster_test

    # Create merged label by sensor (average target class for all events belonging to the same instance)
    unique_inst_id = np.sort(np.unique(id_test))
    y_true_merged = [round(np.average(y_true[id_test == id])) for id in unique_inst_id]

    id_list = []
    y_list = []

    # Iterate through sensors
    for sens in sensors_sorted:

        # Extract X and id for the tried sensor
        X_test_sens = X_test[sens_test == sens]
        id_test_sens = id_test[sens_test == sens]

        # Predict
        y_test_sens = best_models_mc[sens].predict(X_test_sens)

        id_list.extend(id_test_sens)
        y_list.extend(y_test_sens)


    id_array = np.array(id_list)
    y_array = np.array(y_list)
    y_array_merged = [round(np.average(y_array[id_array == id])+0.01) for id in unique_inst_id]
    y_test_merged = np.zeros(len(y_array_merged))
    y_test_merged[np.array(y_array_merged) > 5] = 1



    print(f'Accuracy for case: {case}: {accuracy_score(y_true_merged, y_test_merged)}')
    # print(confusion_matrix(y_true_merged, y_test_merged))
    # print(recall_score(y_true_merged, y_test_merged))

Accuracy for case: A: 0.8146067415730337
Accuracy for case: B: 0.9039704524469068
Accuracy for case: C: 0.7497725204731575


In [20]:
np.average(results_mc)

0.9461251944303513